In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from datasets import load_dataset
# from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch.nn import functional as F
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

torch.manual_seed(42)

In [3]:
dataset = load_dataset("Verah/JParaCrawl-Filtered-English-Japanese-Parallel-Corpus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [4]:
model_repo = 'google/mt5-small'
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
# model.encoder.dropout = torch.nn.Dropout(p=0.1)
# model.decoder.dropout = torch.nn.Dropout(p=0.1)
model = model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def encode_str(text, tokenizer, seq_len=20):
  ids = tokenizer.encode(
      text=text,
      return_tensors='pt',
      padding='max_length',
      truncation=True,
      max_length=seq_len)

  return ids.squeeze()

In [6]:
train_dataset = dataset['train']
i = 8
print(train_dataset[i]['english'])
print(train_dataset[i]['japanese'])

It’s like you can enrich it and save money as a result. I’ve watched a lot of videos, mainly on Youtube, of people who say they’re minimalists, and I agree.
Youtubeを中心にミニマリストと言っている方の動画をたくさんみましたが、納得いくもののも多く、不況と少子化を呼ばれる”今の”日本には合っている考え方なのかな、と感じています。


In [7]:
# getting stats about the datset lengths, not necessary to run
ct = 0
for i in range(len(train_dataset)):
  if i % 1000 == 0:
    print(f"number {i}")
  en_num_tokens = len(tokenizer.encode(train_dataset[i]['english']))
  jp_num_tokens = len(tokenizer.encode(train_dataset[i]['japanese']))
  if en_num_tokens <= 20 and jp_num_tokens <= 20:
    ct += 1
print(ct)

number 0
number 1000
number 2000
number 3000
number 4000
number 5000
number 6000
number 7000
number 8000
number 9000
number 10000
number 11000
number 12000
number 13000
number 14000
number 15000
number 16000
number 17000
number 18000
number 19000
number 20000
number 21000
number 22000
number 23000
number 24000
number 25000
number 26000
number 27000
number 28000
number 29000
number 30000
number 31000
number 32000
number 33000
number 34000
number 35000
number 36000
number 37000
number 38000
number 39000
number 40000
number 41000
number 42000
number 43000
number 44000
number 45000
number 46000
number 47000
number 48000
number 49000
number 50000


KeyboardInterrupt: 

In [7]:
# getting rid of every training example with more than 20 tokens in either pair

filtered_dataset = train_dataset.filter(lambda x: len(tokenizer.encode(x['english'])) <= 20 and len(tokenizer.encode(x['japanese'])) <= 20)


Filter:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [8]:
# function for creating token pairs

def get_tok_pair(pair, tokenizer):
    jp_txt, en_txt = pair['japanese'], pair['english']
    jp_encoded = encode_str(jp_txt, tokenizer)
    en_encoded = encode_str(en_txt, tokenizer)
    return {'japanese': jp_encoded, 'english': en_encoded}

In [9]:
# creating splits of the JParaCrawl dataset

tokenized_dataset = filtered_dataset.map(lambda pair: get_tok_pair(pair, tokenizer))
train_test_dev = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_dev['train']
test_dev = train_test_dev['test']
test_dataset = test_dev.train_test_split(test_size=0.5)
dev_dataset = test_dataset['train']
test_dataset = test_dataset['test']

Map:   0%|          | 0/293062 [00:00<?, ? examples/s]

In [11]:
# you don't have to run this one if you just want to do inference, this data isn't added to the test set

from datasets import load_dataset, concatenate_datasets
# adding more tokens to the train dataset, don't run if you just want jparacrawl
opus_dataset = load_dataset("Helsinki-NLP/opus-100", "en-ja")
opus_dataset = opus_dataset['train']

b_dataset = load_dataset("bsd_ja_en")
b_dataset = b_dataset['train']
all_columns = b_dataset.column_names
columns_to_keep = ['en_sentence', 'ja_sentence']
columns_to_remove = [col for col in all_columns if col not in columns_to_keep]
b_dataset = b_dataset.remove_columns(columns_to_remove)


print(b_dataset[0])
print(opus_dataset[0])

opus_dataset = opus_dataset.map(lambda x: {'english': x['translation']['en'], 'japanese': x['translation']['ja']})
opus_dataset = opus_dataset.remove_columns(['translation'])

print(b_dataset[0])
print(opus_dataset[0])

b_dataset = b_dataset.rename_column('en_sentence', 'english')
b_dataset = b_dataset.rename_column('ja_sentence', 'japanese')

added = concatenate_datasets([opus_dataset, b_dataset])
added = added.filter(lambda x: len(tokenizer.encode(x['english'])) <= 20 and len(tokenizer.encode(x['japanese'])) <= 20)
added = added.map(lambda pair: get_tok_pair(pair, tokenizer))
train_dataset = concatenate_datasets([train_dataset, added])
train_dataset = train_dataset.shuffle(seed=42)
print(len(train_dataset))

KeyboardInterrupt: 

In [ ]:
print(tokenized_dataset[0]['japanese'])
print(tokenized_dataset[0]['english'])

In [12]:
def collate_fn(batch):
    # Prepare batches for Japanese and English by directly extracting tensors
    jp_batch = [torch.tensor(item['japanese']) for item in batch]
    en_batch = [torch.tensor(item['english']) for item in batch]

    # Since all tensors are already processed to be the same length, we can directly stack them
    jp_batch = torch.stack(jp_batch)
    en_batch = torch.stack(en_batch)

    return {
        'japanese': jp_batch,
        'english': en_batch
    }

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [13]:
n_epochs = 4
batch_size = 64
print_freq = 50
val_freq = 1000
lr = 1e-3
n_batches = int(np.ceil(len(filtered_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.1)
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=a177509cb721ae5a8a5652806ff37d3d497a3286c24adedb1b791e43a35f343d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
"""
STOP EXECUTING HERE IF YOU JUST WANT TO DO INFERENCE, SKIP NEXT THREE AND SCROLL DOWN TO LOAD CHECKPOINTS INTO THE MODEL
"""

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

In [14]:
# model.train()
train_losses = []
val_losses = []

for epoch in range(n_epochs):
    model.train()
    print(f"Starting epoch {epoch + 1}/{n_epochs}")
    epoch_loss = 0
    for step, batch in tqdm_notebook(enumerate(train_dataloader), total=n_batches):
        # Ensure batch tensors are on the same device as model
        jp_batch = batch['japanese'].to(model.device)
        en_batch = batch['english'].to(model.device)

        # Clear previous gradients
        optimizer.zero_grad()

        # Prepare the inputs to the model
        inputs = {
            'input_ids': jp_batch,
            'labels': en_batch
        }

        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss

        # Backward pass to calculate the gradients
        loss.backward()

        # Update weights
        optimizer.step()
        scheduler.step()

        # Accumulate the loss
        epoch_loss += loss.item()

        # Print loss every print_freq steps
        if (step + 1) % print_freq == 0 or (step + 1) == len(train_dataloader):
            print(f"Step {step + 1}/{len(train_dataloader)}, Loss: {loss.item()}")

        # Perform validation every val_freq steps
        if (step + 1) % val_freq == 0 or (step + 1) == len(train_dataloader):
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for val_batch in val_dataloader:
                    jp_val_batch = val_batch['japanese'].to(model.device)
                    en_val_batch = val_batch['english'].to(model.device)

                    val_inputs = {
                        'input_ids': jp_val_batch,
                        'labels': en_val_batch
                    }

                    val_outputs = model(**val_inputs)
                    val_loss += val_outputs.loss.item()

            val_loss /= len(val_dataloader)
            val_losses.append(val_loss)
            print(f"Validation Loss: {val_loss}")
            model.train()

    val_translations = []
    val_references = []
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for batch in val_dataloader:
            jp_batch = batch['japanese'].to(model.device)
            en_batch = batch['english'].to(model.device)

            # Specifying max_new_tokens to control the generation length
            model_out = model.generate(jp_batch, max_new_tokens=20)
            translations = tokenizer.batch_decode(model_out, skip_special_tokens=True)

            val_translations.extend(translations)
            val_references.extend(tokenizer.batch_decode(en_batch, skip_special_tokens=True))

    # Calculate the BLEU score
    from nltk.translate.bleu_score import corpus_bleu
    bleu_score = corpus_bleu([[ref.split()] for ref in val_references], [trans.split() for trans in val_translations])
    print(f"BLEU score: {bleu_score}")

    # Calculate the ROUGE score
    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, trans) for ref, trans in zip(val_references, val_translations)]
    # Calculating average scores might be useful for summary
    average_rouge_scores = {key: sum(score[key].fmeasure for score in rouge_scores) / len(rouge_scores) for key in rouge_scores[0]}
    print(f"Average ROUGE scores: {average_rouge_scores}")


    # Average loss for the epoch
    epoch_average_loss = epoch_loss / len(train_dataloader)
    train_losses.append(epoch_average_loss)
    print(f"Epoch {epoch + 1} average loss: {epoch_average_loss}")

    # Save the model checkpoint
    checkpoint_path = f"sample_data/model_checkpoint_epoch_{epoch + 1}.pt"
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Model checkpoint saved at: {checkpoint_path}")

Starting epoch 1/4


<ipython-input-14-cf9da4328bc9>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in tqdm_notebook(enumerate(train_dataloader), total=n_batches):


  0%|          | 0/4580 [00:00<?, ?it/s]

Step 50/17999, Loss: 37.75599670410156
Step 100/17999, Loss: 31.774078369140625
Step 150/17999, Loss: 23.520244598388672
Step 200/17999, Loss: 19.18842315673828
Step 250/17999, Loss: 14.661415100097656
Step 300/17999, Loss: 11.146191596984863
Step 350/17999, Loss: 8.342195510864258
Step 400/17999, Loss: 6.25309419631958
Step 450/17999, Loss: 4.784905910491943
Step 500/17999, Loss: 4.361782073974609
Step 550/17999, Loss: 4.0175886154174805
Step 600/17999, Loss: 3.776398181915283
Step 650/17999, Loss: 3.7458724975585938
Step 700/17999, Loss: 3.704589366912842
Step 750/17999, Loss: 3.233491897583008
Step 800/17999, Loss: 3.2503738403320312
Step 850/17999, Loss: 3.2600464820861816
Step 900/17999, Loss: 3.1768686771392822
Step 950/17999, Loss: 2.611356258392334
Step 1000/17999, Loss: 2.954619884490967
Validation Loss: 2.7991591530595805
Step 1050/17999, Loss: 2.736064910888672
Step 1100/17999, Loss: 2.4221959114074707
Step 1150/17999, Loss: 2.4170069694519043
Step 1200/17999, Loss: 2.625746

<ipython-input-14-cf9da4328bc9>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in tqdm_notebook(enumerate(train_dataloader), total=n_batches):


...

Step 50/17999, Loss: 1.0894372463226318
Step 100/17999, Loss: 1.1208794116973877
Step 150/17999, Loss: 1.0598504543304443
Step 200/17999, Loss: 1.1563024520874023
Step 250/17999, Loss: 0.9737884402275085
Step 300/17999, Loss: 1.2616173028945923
Step 350/17999, Loss: 1.1172802448272705
Step 400/17999, Loss: 1.186736822128296
Step 450/17999, Loss: 1.0485726594924927
Step 500/17999, Loss: 1.2089531421661377
Step 550/17999, Loss: 1.3157787322998047
Step 600/17999, Loss: 1.33774733543396
Step 650/17999, Loss: 1.4382455348968506
Step 700/17999, Loss: 1.1524758338928223
Step 750/17999, Loss: 1.021020770072937
Step 800/17999, Loss: 1.0631849765777588
Step 850/17999, Loss: 0.9634186625480652
Step 900/17999, Loss: 1.3957089185714722
Step 950/17999, Loss: 1.0543917417526245
Step 1000/17999, Loss: 1.2186341285705566
Validation Loss: 0.9291494545978229
Step 1050/17999, Loss: 1.1561455726623535
Step 1100/17999, Loss: 1.3063559532165527
Step 1150/17999, Loss: 1.2271053791046143
Step 1200/17999, Loss:

  0%|          | 0/4580 [00:00<?, ?it/s]

Step 50/17999, Loss: 1.0152145624160767
Step 100/17999, Loss: 1.1247520446777344
Step 150/17999, Loss: 1.0574299097061157
Step 200/17999, Loss: 1.2331397533416748
Step 250/17999, Loss: 1.1963095664978027
Step 300/17999, Loss: 1.1635472774505615
Step 350/17999, Loss: 0.9590904116630554
Step 400/17999, Loss: 1.2616870403289795
Step 450/17999, Loss: 1.171189308166504
Step 500/17999, Loss: 1.2094204425811768
Step 550/17999, Loss: 1.1166446208953857
Step 600/17999, Loss: 1.171475887298584
Step 650/17999, Loss: 1.1054893732070923
Step 700/17999, Loss: 1.2936134338378906
Step 750/17999, Loss: 1.2441859245300293
Step 800/17999, Loss: 1.0731912851333618
Step 850/17999, Loss: 1.1106499433517456
Step 900/17999, Loss: 1.2349936962127686
Step 950/17999, Loss: 1.0691736936569214
Step 1000/17999, Loss: 1.0937573909759521
Validation Loss: 0.9291725746929386
Step 1050/17999, Loss: 1.0576622486114502
Step 1100/17999, Loss: 1.1349653005599976
Step 1150/17999, Loss: 1.0870187282562256
Step 1200/17999, Los

  0%|          | 0/4580 [00:00<?, ?it/s]

Step 50/17999, Loss: 1.0839096307754517
Step 100/17999, Loss: 1.2424005270004272
Step 150/17999, Loss: 1.1760531663894653
Step 200/17999, Loss: 1.1222537755966187
Step 250/17999, Loss: 1.0669138431549072
Step 300/17999, Loss: 1.2814929485321045
Step 350/17999, Loss: 1.3309956789016724
Step 400/17999, Loss: 1.0862184762954712
Step 450/17999, Loss: 1.1631083488464355
Step 500/17999, Loss: 1.1930458545684814
Step 550/17999, Loss: 1.1827632188796997
Step 600/17999, Loss: 1.188878059387207
Step 650/17999, Loss: 1.0971125364303589
Step 700/17999, Loss: 1.1261578798294067
Step 750/17999, Loss: 1.2099621295928955
Step 800/17999, Loss: 1.2022292613983154
Step 850/17999, Loss: 1.1016372442245483
Step 900/17999, Loss: 1.2928727865219116
Step 950/17999, Loss: 1.242668867111206
Step 1000/17999, Loss: 1.2322975397109985
Validation Loss: 0.9291633841252223
Step 1050/17999, Loss: 1.2616215944290161
Step 1100/17999, Loss: 1.3016960620880127
Step 1150/17999, Loss: 1.4212563037872314
Step 1200/17999, Los

In [ ]:
model.eval()
val_loss = 0
with torch.no_grad():
    for val_batch in val_dataloader:
        jp_val_batch = val_batch['japanese'].to(model.device)
        en_val_batch = val_batch['english'].to(model.device)

        val_inputs = {
            'input_ids': jp_val_batch,
            'labels': en_val_batch
        }

        val_outputs = model(**val_inputs)
        val_loss += val_outputs.loss.item()

    val_loss /= len(val_dataloader)
    #val_losses.append(val_loss)
    print(f"Validation Loss: {val_loss}")

In [34]:
# LOAD MODEL FROM PATH FOR INFERENCE

model_path = '/content/drive/My Drive/ModelCheckpoints/model_checkpoint_epoch_3_seeded_no_dropout.pt' # replace with path to unzipped checkpoints
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [30]:
model.eval()

input_sentence = '時期は2022年1月頃を予定しています。' # change to your desired sentence for inference!
token_ids = tokenizer.encode(input_sentence, return_tensors='pt').cuda()
model_out = model.generate(token_ids)
print(model_out)
print(tokenizer.decode(model_out[0]))

tensor([[    0,   486,   259, 39050,   339, 31499,   285,   344,   259,  3829,
         26221,   281, 60352,   260,     1]], device='cuda:0')
<pad> The duration is scheduled at January 2021 in 2022.</s>


In [35]:
test_translations = []
test_references = []
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for batch in test_dataloader:
        jp_batch = batch['japanese'].to(model.device)
        en_batch = batch['english'].to(model.device)

        # Specifying max_new_tokens to control the generation length
        model_out = model.generate(jp_batch, max_new_tokens=20)
        translations = tokenizer.batch_decode(model_out, skip_special_tokens=True)

        test_translations.extend(translations)
        test_references.extend(tokenizer.batch_decode(en_batch, skip_special_tokens=True))

# Calculate the BLEU score
from nltk.translate.bleu_score import corpus_bleu
bleu_score = corpus_bleu([[ref.split()] for ref in test_references], [trans.split() for trans in test_translations])
print(f"BLEU score: {bleu_score}")

# Calculate the ROUGE score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(ref, trans) for ref, trans in zip(test_references, test_translations)]
# Calculating average scores might be useful for summary
average_rouge_scores = {key: sum(score[key].fmeasure for score in rouge_scores) / len(rouge_scores) for key in rouge_scores[0]}
print(f"Average ROUGE scores: {average_rouge_scores}")


BLEU score: 0.3621292614694309
Average ROUGE scores: {'rouge1': 0.7080705148417517, 'rouge2': 0.5014807463955337, 'rougeL': 0.6794601847718821}


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
import os
os.makedirs('/content/drive/My Drive/ModelCheckpoints', exist_ok=True)
import shutil
for i in range(1, n_epochs+1):
  source = f'/content/sample_data/model_checkpoint_epoch_{i}.pt'
  destination = f'/content/drive/My Drive/ModelCheckpoints/model_checkpoint_epoch_{i}_more_data.pt'
  shutil.copy(source, destination)